# 🎯 08. Ultimate Integration - Universal AI Assistant

## 🌟 Day 3의 모든 것을 하나로!

**3가지 전문 Agent를 단일 Supervisor가 지휘하는 통합 AI 시스템**

---

## 🏗️ 시스템 아키텍처

```
사용자 (Gradio 채팅 UI)
    ↓
🎯 Top Supervisor (LLM 기반 지능형 라우팅)
    ├─→ 🛒 Customer Support Agent
    │    └─ 주문 조회, 환불 처리, 재고 확인
    │
    ├─→ 📚 Research Agent  
    │    └─ Wikipedia 검색 → 분석 → 마크다운 보고서
    │
    └─→ 💾 Text2SQL Agent
         └─ 자연어 → SQL 생성 → DB 쿼리 → 결과 포맷팅
```

**핵심:** 사용자는 하나의 채팅창만 사용! Supervisor가 자동으로 적절한 Agent 선택!

---

## 📚 통합된 Day 3 개념

| 노트북 | 적용 개념 |
|--------|----------|
| **03** | StateGraph, Command, 조건부 라우팅 |
| **04** | ReAct 패턴, Tool 통합 |
| **05** | **Supervisor 패턴** (핵심!) |
| **06** | 외부 도구 통합 (Wikipedia, SQL) |
| **07** | Memory & Persistence (선택) |

---

## 🎮 사용 예시

```python
사용자: "주문 12345 어디까지 왔어?"
→ 🛒 Customer Support Agent 자동 선택
→ DB 조회 → "배송중 (서울 허브)"

사용자: "블록체인 기술 조사해줘"
→ 📚 Research Agent 자동 선택
→ Wikipedia → 보고서 작성 → 파일 저장

사용자: "매출 상위 5개 상품 알려줘"
→ 💾 Text2SQL Agent 자동 선택
→ SQL 생성 → 실행 → 테이블 출력
```

**마지막 피날레, 시작합니다!** 🚀

---

# Part 0: 환경 설정 🔧

## 💡 이 Part에서 하는 것
- LLM 로딩 (EXAONE 3.5)
- 필수 패키지 설치
- 공통 State 정의

**예상 시간**: 3-5분

In [ ]:
# 패키지 설치
!pip install -q langgraph langchain-core langchain-huggingface
!pip install -q transformers torch accelerate
!pip install -q gradio wikipedia

print("✅ 패키지 설치 완료!")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline
from typing import TypedDict, List, Literal
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
import json
import sqlite3
import os
from datetime import datetime

print("✅ Import 완료!")

In [ ]:
print("🔄 EXAONE 3.5 모델 로딩 중... (~2분)")

tokenizer = AutoTokenizer.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
    trust_remote_code=True,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200,
    temperature=0.7,
    return_full_text=False
)

llm = HuggingFacePipeline(pipeline=pipe)

print(f"✅ 모델 로딩 완료! ({'GPU' if torch.cuda.is_available() else 'CPU'} 사용)")

### State 정의

모든 Agent가 공유하는 State입니다.

In [ ]:
class UniversalState(TypedDict):
    query: str          # 사용자 질문
    response: str       # 최종 답변
    agent_used: str     # 어떤 Agent가 처리했는지
    metadata: dict      # 추가 정보 (파일명, SQL 등)

print("✅ State 정의 완료!")

---

# Part 1: 데이터 & 도구 준비 🛠️

## 💡 이 Part에서 하는 것
- 샘플 데이터 생성 (orders.json, products.db)
- 3가지 Agent가 사용할 도구 함수들 구현
- Wikipedia API 연동

**예상 시간**: 5분

### 1.1 Customer Support 데이터 (orders.json)

In [ ]:
# 주문 데이터 생성
orders_data = {
    "12345": {
        "status": "배송중",
        "location": "서울 허브",
        "expected": "2025-10-28",
        "product": "무선 키보드",
        "price": 89000,
        "customer": "김철수"
    },
    "12346": {
        "status": "배송 완료",
        "location": "고객 수령",
        "expected": "2025-10-25",
        "product": "게이밍 마우스",
        "price": 65000,
        "customer": "이영희"
    },
    "12347": {
        "status": "준비중",
        "location": "창고",
        "expected": "2025-10-30",
        "product": "노트북",
        "price": 1290000,
        "customer": "박민수"
    },
    "12348": {
        "status": "환불 완료",
        "location": "환불 처리됨",
        "expected": "-",
        "product": "모니터",
        "price": 450000,
        "customer": "최수진",
        "refund_reason": "불량 화소",
        "refund_date": "2025-10-24"
    }
}

# 파일 저장
with open('orders.json', 'w', encoding='utf-8') as f:
    json.dump(orders_data, f, ensure_ascii=False, indent=2)

print("✅ orders.json 생성 완료!")
print(f"   총 {len(orders_data)}개 주문 데이터")

### 1.2 Text2SQL 데이터 (products.db)

In [ ]:
# SQLite 데이터베이스 생성
conn = sqlite3.connect('products.db')
cursor = conn.cursor()

# 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS products (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    category TEXT NOT NULL,
    price INTEGER NOT NULL,
    stock INTEGER NOT NULL,
    sales INTEGER NOT NULL
)
''')

# 샘플 데이터 삽입
products = [
    (1, '무선 키보드', '주변기기', 89000, 45, 230),
    (2, '게이밍 마우스', '주변기기', 65000, 32, 189),
    (3, '노트북', '컴퓨터', 1290000, 12, 78),
    (4, '모니터 27인치', '디스플레이', 450000, 8, 156),
    (5, 'USB-C 허브', '주변기기', 35000, 67, 412),
    (6, '외장 SSD 1TB', '저장장치', 120000, 23, 95),
    (7, '웹캠 Full HD', '주변기기', 78000, 19, 134),
    (8, '기계식 키보드', '주변기기', 145000, 15, 87),
    (9, '노이즈캔슬링 헤드폰', '오디오', 320000, 11, 62),
    (10, '태블릿 10인치', '컴퓨터', 580000, 9, 45)
]

cursor.executemany('INSERT OR REPLACE INTO products VALUES (?,?,?,?,?,?)', products)
conn.commit()
conn.close()

print("✅ products.db 생성 완료!")
print(f"   총 {len(products)}개 상품 등록")

### 1.3 도구 함수 구현

3가지 Agent가 사용할 도구들입니다.

In [ ]:
# ===== Customer Support 도구 =====

def search_order(order_id: str) -> str:
    """주문 정보를 조회합니다"""
    try:
        with open('orders.json', 'r', encoding='utf-8') as f:
            orders = json.load(f)
        
        if order_id in orders:
            order = orders[order_id]
            return f"""주문번호: {order_id}
상품: {order['product']}
상태: {order['status']}
위치: {order['location']}
예상 도착: {order['expected']}
금액: {order['price']:,}원
고객: {order['customer']}"""
        else:
            return f"주문번호 {order_id}를 찾을 수 없습니다."
    except Exception as e:
        return f"오류 발생: {str(e)}"


def process_refund(order_id: str, reason: str) -> str:
    """환불을 처리합니다"""
    try:
        with open('orders.json', 'r', encoding='utf-8') as f:
            orders = json.load(f)
        
        if order_id not in orders:
            return f"주문번호 {order_id}를 찾을 수 없습니다."
        
        orders[order_id]['status'] = '환불 완료'
        orders[order_id]['refund_reason'] = reason
        orders[order_id]['refund_date'] = datetime.now().strftime("%Y-%m-%d")
        
        with open('orders.json', 'w', encoding='utf-8') as f:
            json.dump(orders, f, ensure_ascii=False, indent=2)
        
        return f"환불 처리 완료: {order_id} (사유: {reason})"
    except Exception as e:
        return f"오류 발생: {str(e)}"


def check_inventory(product_name: str) -> str:
    """재고를 확인합니다"""
    try:
        conn = sqlite3.connect('products.db')
        cursor = conn.cursor()
        cursor.execute(
            "SELECT name, stock, price FROM products WHERE name LIKE ?",
            (f"%{product_name}%",)
        )
        result = cursor.fetchone()
        conn.close()
        
        if result:
            return f"{result[0]}: 재고 {result[1]}개, 가격 {result[2]:,}원"
        else:
            return f"'{product_name}' 상품을 찾을 수 없습니다."
    except Exception as e:
        return f"오류 발생: {str(e)}"


print("✅ Customer Support 도구 정의 완료!")

In [ ]:
# ===== Research 도구 =====

import wikipedia
wikipedia.set_lang("ko")  # 한국어 설정

def search_wikipedia(query: str) -> str:
    """Wikipedia에서 정보를 검색합니다"""
    try:
        # 검색어로 페이지 찾기
        page = wikipedia.page(query, auto_suggest=True)
        
        # 요약 반환 (처음 500자)
        summary = page.summary[:500]
        return f"**{page.title}**\n\n{summary}..."
    except wikipedia.exceptions.DisambiguationError as e:
        # 동음이의어인 경우 첫 번째 옵션 사용
        return search_wikipedia(e.options[0])
    except wikipedia.exceptions.PageError:
        return f"'{query}'에 대한 Wikipedia 페이지를 찾을 수 없습니다."
    except Exception as e:
        return f"오류 발생: {str(e)}"


def save_markdown(filename: str, content: str) -> str:
    """마크다운 파일을 저장합니다"""
    try:
        os.makedirs('reports', exist_ok=True)
        filepath = f"reports/{filename}"
        
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(content)
        
        return f"보고서 저장 완료: {filepath}"
    except Exception as e:
        return f"오류 발생: {str(e)}"


print("✅ Research 도구 정의 완료!")

In [ ]:
# ===== Text2SQL 도구 =====

def execute_sql(query: str) -> str:
    """SQL 쿼리를 실행합니다"""
    try:
        conn = sqlite3.connect('products.db')
        cursor = conn.cursor()
        cursor.execute(query)
        
        # SELECT 쿼리인 경우
        if query.strip().upper().startswith('SELECT'):
            results = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]
            conn.close()
            
            if not results:
                return "결과 없음"
            
            # 테이블 형식으로 포맷팅
            output = "\n".join([" | ".join(map(str, row)) for row in results])
            return f"{' | '.join(columns)}\n{'-'*50}\n{output}"
        else:
            # INSERT, UPDATE, DELETE 등
            conn.commit()
            conn.close()
            return "쿼리 실행 완료"
    except Exception as e:
        return f"SQL 오류: {str(e)}"


def get_schema() -> str:
    """데이터베이스 스키마를 반환합니다"""
    return """테이블: products
컬럼:
  - id: INTEGER (기본 키)
  - name: TEXT (상품명)
  - category: TEXT (카테고리)
  - price: INTEGER (가격)
  - stock: INTEGER (재고)
  - sales: INTEGER (판매량)

예시 쿼리:
  SELECT name, price FROM products WHERE category = '주변기기'
  SELECT name, sales FROM products ORDER BY sales DESC LIMIT 5"""


print("✅ Text2SQL 도구 정의 완료!")

### 1.4 도구 테스트

In [ ]:
# 간단한 테스트
print("🔍 도구 테스트:\n")

print("[Customer Support]")
print(search_order("12345"))
print()

print("[Research]")
print(search_wikipedia("인공지능")[:200] + "...")
print()

print("[Text2SQL]")
print(execute_sql("SELECT name, price FROM products LIMIT 3"))
print()

print("✅ 모든 도구가 정상 작동합니다!")

---

# Part 2: Customer Support Agent 🛒

## 💡 이 Part에서 하는 것
- 주문/환불/재고 전문 Agent 구현
- **Handoff 패턴** 적용 (05번 노트북)
- 3개 전문가 노드 + 라우터 구조

**예상 시간**: 7분

### 2.1 State 정의

In [ ]:
class CustomerSupportState(TypedDict):
    query: str          # 사용자 질문
    response: str       # 최종 답변
    agent_path: List[str]  # 거쳐간 Agent들

print("✅ CustomerSupportState 정의 완료!")

### 2.2 Router 노드

LLM을 사용해 사용자 질문을 분석하고 적절한 전문가에게 라우팅합니다.

In [ ]:
def cs_router(state: CustomerSupportState) -> Command:
    """사용자 질문을 분석해서 적절한 전문가 Agent로 라우팅"""
    query = state['query']
    
    # LLM에게 라우팅 결정 요청
    prompt = f"""다음 고객 질문을 보고 어떤 전문가가 처리해야 할지 결정하세요.

고객 질문: {query}

전문가 목록:
- order_agent: 주문 조회, 배송 상태 확인
- refund_agent: 환불, 취소, 반품
- inventory_agent: 재고 확인, 상품 정보

답변은 반드시 전문가 이름만 출력하세요 (order_agent, refund_agent, inventory_agent 중 하나)."""
    
    response = llm.invoke(prompt)
    decision = response.strip().lower()
    
    # 간단한 키워드 기반 폴백
    if 'order' not in decision and 'refund' not in decision and 'inventory' not in decision:
        if any(word in query for word in ['주문', '배송', '어디']):
            decision = 'order_agent'
        elif any(word in query for word in ['환불', '취소', '반품']):
            decision = 'refund_agent'
        elif any(word in query for word in ['재고', '있어', '가격']):
            decision = 'inventory_agent'
        else:
            decision = 'order_agent'  # 기본값
    
    # 전문가 이름 추출
    if 'order' in decision:
        next_agent = 'order_agent'
    elif 'refund' in decision:
        next_agent = 'refund_agent'
    elif 'inventory' in decision:
        next_agent = 'inventory_agent'
    else:
        next_agent = 'order_agent'
    
    print(f"🎯 Router: {next_agent} 선택")
    return Command(goto=next_agent, update={'agent_path': [next_agent]})

print("✅ Router 노드 정의 완료!")

### 2.3 전문가 Agent 노드들

In [ ]:
def order_agent(state: CustomerSupportState) -> CustomerSupportState:
    """주문 조회 전문가"""
    query = state['query']
    
    # 주문번호 추출 (간단한 패턴)
    import re
    order_id_match = re.search(r'\d{5}', query)
    
    if order_id_match:
        order_id = order_id_match.group()
        result = search_order(order_id)
    else:
        result = "주문번호를 찾을 수 없습니다. 5자리 숫자를 입력해주세요."
    
    print(f"📦 Order Agent 실행 완료")
    return {**state, 'response': result}


def refund_agent(state: CustomerSupportState) -> CustomerSupportState:
    """환불 처리 전문가"""
    query = state['query']
    
    # 주문번호 추출
    import re
    order_id_match = re.search(r'\d{5}', query)
    
    if order_id_match:
        order_id = order_id_match.group()
        # 사유 추출 (간단한 버전)
        reason = "고객 요청"
        if '불량' in query:
            reason = "제품 불량"
        elif '마음' in query or '변심' in query:
            reason = "단순 변심"
        
        result = process_refund(order_id, reason)
    else:
        result = "환불할 주문번호를 입력해주세요."
    
    print(f"💰 Refund Agent 실행 완료")
    return {**state, 'response': result}


def inventory_agent(state: CustomerSupportState) -> CustomerSupportState:
    """재고 확인 전문가"""
    query = state['query']
    
    # 상품명 추출 (간단한 키워드 매칭)
    keywords = ['키보드', '마우스', '노트북', '모니터', '헤드폰', '태블릿', 'SSD', '웹캠']
    product_name = None
    
    for keyword in keywords:
        if keyword in query:
            product_name = keyword
            break
    
    if product_name:
        result = check_inventory(product_name)
    else:
        result = "상품명을 명확히 입력해주세요. (예: 키보드, 마우스, 노트북 등)"
    
    print(f"📊 Inventory Agent 실행 완료")
    return {**state, 'response': result}


print("✅ 전문가 Agent 노드들 정의 완료!")

### 2.4 Customer Support Graph 구축

In [ ]:
# Graph 생성
cs_graph = StateGraph(CustomerSupportState)

# 노드 추가
cs_graph.add_node("router", cs_router)
cs_graph.add_node("order_agent", order_agent)
cs_graph.add_node("refund_agent", refund_agent)
cs_graph.add_node("inventory_agent", inventory_agent)

# 엣지 설정
cs_graph.add_edge(START, "router")
cs_graph.add_edge("order_agent", END)
cs_graph.add_edge("refund_agent", END)
cs_graph.add_edge("inventory_agent", END)

# 컴파일
customer_support_app = cs_graph.compile()

print("✅ Customer Support Graph 구축 완료!")

### 2.5 Customer Support Agent 테스트

In [ ]:
# 테스트 1: 주문 조회
print("=== 테스트 1: 주문 조회 ===")
result1 = customer_support_app.invoke({
    'query': '주문 12345 어디까지 왔어?',
    'response': '',
    'agent_path': []
})
print(f"\n답변:\n{result1['response']}\n")

# 테스트 2: 재고 확인
print("=== 테스트 2: 재고 확인 ===")
result2 = customer_support_app.invoke({
    'query': '키보드 재고 있어?',
    'response': '',
    'agent_path': []
})
print(f"\n답변:\n{result2['response']}\n")

print("✅ Customer Support Agent 작동 확인!")

---

# Part 3: Research Agent 📚

## 💡 이 Part에서 하는 것
- Wikipedia 검색 → 분석 → 보고서 작성 Agent
- **Sequential Flow** 패턴 (검색 → 분석 → 작성)
- 마크다운 파일 자동 저장

**예상 시간**: 7분

### 3.1 State 정의

In [ ]:
class ResearchState(TypedDict):
    query: str              # 연구 주제
    wiki_content: str       # Wikipedia 검색 결과
    analysis: str           # LLM 분석 결과
    report: str             # 최종 보고서 (마크다운)
    filename: str           # 저장된 파일명

print("✅ ResearchState 정의 완료!")

### 3.2 Research Agent 노드들

In [ ]:
def search_node(state: ResearchState) -> ResearchState:
    """Wikipedia에서 정보 검색"""
    query = state['query']
    print(f"🔍 Wikipedia 검색 중: {query}")
    
    wiki_content = search_wikipedia(query)
    
    return {**state, 'wiki_content': wiki_content}


def analyze_node(state: ResearchState) -> ResearchState:
    """LLM으로 내용 분석"""
    wiki_content = state['wiki_content']
    print(f"🤔 LLM 분석 중...")
    
    prompt = f"""다음 Wikipedia 내용을 분석하고 핵심 포인트 3가지를 추출하세요.

내용:
{wiki_content}

다음 형식으로 답변하세요:
1. [첫 번째 핵심 포인트]
2. [두 번째 핵심 포인트]
3. [세 번째 핵심 포인트]"""
    
    analysis = llm.invoke(prompt)
    
    return {**state, 'analysis': analysis}


def write_report_node(state: ResearchState) -> ResearchState:
    """마크다운 보고서 작성"""
    query = state['query']
    wiki_content = state['wiki_content']
    analysis = state['analysis']
    print(f"📝 보고서 작성 중...")
    
    # 마크다운 보고서 생성
    report = f"""# 리서치 보고서: {query}

**생성일**: {datetime.now().strftime("%Y-%m-%d %H:%M")}

---

## 📖 Wikipedia 요약

{wiki_content}

---

## 🎯 핵심 포인트 (LLM 분석)

{analysis}

---

## 📌 결론

위 내용을 바탕으로 '{query}'에 대한 기본적인 이해를 할 수 있습니다.
더 자세한 내용은 Wikipedia 원문을 참고하세요.

*본 보고서는 AI가 자동 생성했습니다.*
"""
    
    # 파일 저장
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"research_{timestamp}.md"
    save_result = save_markdown(filename, report)
    
    print(f"✅ {save_result}")
    
    return {**state, 'report': report, 'filename': filename}


print("✅ Research Agent 노드들 정의 완료!")

### 3.3 Research Graph 구축

In [ ]:
# Graph 생성
research_graph = StateGraph(ResearchState)

# 노드 추가
research_graph.add_node("search", search_node)
research_graph.add_node("analyze", analyze_node)
research_graph.add_node("write_report", write_report_node)

# Sequential Flow
research_graph.add_edge(START, "search")
research_graph.add_edge("search", "analyze")
research_graph.add_edge("analyze", "write_report")
research_graph.add_edge("write_report", END)

# 컴파일
research_app = research_graph.compile()

print("✅ Research Graph 구축 완료!")

### 3.4 Research Agent 테스트

In [ ]:
# 테스트: 블록체인 리서치
print("=== Research Agent 테스트 ===")
print("주제: 블록체인\n")

result = research_app.invoke({
    'query': '블록체인',
    'wiki_content': '',
    'analysis': '',
    'report': '',
    'filename': ''
})

print(f"\n📄 생성된 파일: reports/{result['filename']}")
print(f"\n보고서 미리보기 (처음 300자):\n")
print(result['report'][:300] + "...\n")

print("✅ Research Agent 작동 확인!")

---

# Part 4: Text2SQL Agent 💾

## 💡 이 Part에서 하는 것
- 자연어 질문 → SQL 쿼리 변환
- **ReAct 패턴** 적용 (04번 노트북)
- 스키마 기반 SQL 생성 → 실행 → 포맷팅

**예상 시간**: 8분

### 4.1 State 정의

In [ ]:
class Text2SQLState(TypedDict):
    query: str              # 사용자 질문
    sql_query: str          # 생성된 SQL
    result: str             # 쿼리 실행 결과
    formatted_response: str # 사람이 읽기 쉬운 형식
    error: str              # 에러 메시지 (있다면)

print("✅ Text2SQLState 정의 완료!")

### 4.2 Text2SQL Agent 노드들

In [ ]:
def generate_sql_node(state: Text2SQLState) -> Text2SQLState:
    """자연어 질문을 SQL로 변환"""
    query = state['query']
    print(f"🔄 SQL 생성 중: {query}")
    
    schema = get_schema()
    
    prompt = f"""다음 데이터베이스 스키마를 참고하여 자연어 질문을 SQL 쿼리로 변환하세요.

{schema}

질문: {query}

SQL 쿼리만 출력하세요 (주석이나 설명 없이)."""
    
    sql_response = llm.invoke(prompt)
    
    # SQL 쿼리 추출 (간단한 클리닝)
    sql_query = sql_response.strip()
    # 마크다운 코드 블록 제거
    if '```' in sql_query:
        import re
        match = re.search(r'```(?:sql)?\s*([^`]+)```', sql_query)
        if match:
            sql_query = match.group(1).strip()
    
    # 세미콜론 제거 (SQLite 실행 시 문제 방지)
    sql_query = sql_query.rstrip(';')
    
    print(f"📝 생성된 SQL: {sql_query}")
    
    return {**state, 'sql_query': sql_query}


def execute_sql_node(state: Text2SQLState) -> Text2SQLState:
    """SQL 쿼리 실행"""
    sql_query = state['sql_query']
    print(f"⚙️ SQL 실행 중...")
    
    try:
        result = execute_sql(sql_query)
        print(f"✅ 실행 완료")
        return {**state, 'result': result, 'error': ''}
    except Exception as e:
        error_msg = f"SQL 실행 오류: {str(e)}"
        print(f"❌ {error_msg}")
        return {**state, 'result': '', 'error': error_msg}


def format_response_node(state: Text2SQLState) -> Text2SQLState:
    """결과를 사람이 읽기 쉬운 형식으로 변환"""
    query = state['query']
    result = state['result']
    error = state['error']
    print(f"📊 결과 포맷팅 중...")
    
    if error:
        formatted = f"⚠️ 오류가 발생했습니다:\n{error}"
    else:
        formatted = f"""📊 질문: {query}

결과:
{result}"""
    
    return {**state, 'formatted_response': formatted}


print("✅ Text2SQL Agent 노드들 정의 완료!")

### 4.3 Text2SQL Graph 구축

In [ ]:
# Graph 생성
text2sql_graph = StateGraph(Text2SQLState)

# 노드 추가
text2sql_graph.add_node("generate_sql", generate_sql_node)
text2sql_graph.add_node("execute_sql", execute_sql_node)
text2sql_graph.add_node("format_response", format_response_node)

# Sequential Flow
text2sql_graph.add_edge(START, "generate_sql")
text2sql_graph.add_edge("generate_sql", "execute_sql")
text2sql_graph.add_edge("execute_sql", "format_response")
text2sql_graph.add_edge("format_response", END)

# 컴파일
text2sql_app = text2sql_graph.compile()

print("✅ Text2SQL Graph 구축 완료!")

### 4.4 Text2SQL Agent 테스트

In [ ]:
# 테스트 1: 매출 상위 5개 상품
print("=== 테스트 1: 매출 상위 상품 ===")
result1 = text2sql_app.invoke({
    'query': '매출 상위 5개 상품 알려줘',
    'sql_query': '',
    'result': '',
    'formatted_response': '',
    'error': ''
})
print(f"\n{result1['formatted_response']}\n")

# 테스트 2: 주변기기 카테고리 상품
print("=== 테스트 2: 카테고리 필터링 ===")
result2 = text2sql_app.invoke({
    'query': '주변기기 카테고리 상품들의 평균 가격은?',
    'sql_query': '',
    'result': '',
    'formatted_response': '',
    'error': ''
})
print(f"\n{result2['formatted_response']}\n")

print("✅ Text2SQL Agent 작동 확인!")

---

# Part 5: Top Supervisor 통합 🎯

## 💡 이 Part에서 하는 것
- 3가지 전문 Agent를 하나로 통합!
- **Supervisor 패턴** (05번 노트북 핵심 개념)
- LLM 기반 지능형 라우팅

**예상 시간**: 10분

### 5.1 Top Supervisor 구현

사용자 질문을 분석해서 적절한 Agent를 선택합니다.

In [ ]:
def top_supervisor(state: UniversalState) -> Command:
    """사용자 질문을 분석하고 적절한 Agent로 라우팅"""
    query = state['query']
    print(f"\n🎯 Top Supervisor 분석 중...")
    print(f"   질문: {query}")
    
    # LLM에게 라우팅 결정 요청
    prompt = f"""다음 사용자 질문을 보고 어떤 전문 Agent가 처리해야 할지 결정하세요.

질문: {query}

Agent 목록:
- customer_support: 주문 조회, 환불 처리, 재고 확인 (전자상거래 고객 지원)
- research: Wikipedia 검색, 리서치 보고서 작성 (정보 조사)
- text2sql: 데이터베이스 쿼리, 통계 분석, 상품 데이터 조회 (SQL 쿼리)

답변은 반드시 Agent 이름만 출력하세요 (customer_support, research, text2sql 중 하나)."""
    
    response = llm.invoke(prompt).strip().lower()
    
    # 간단한 키워드 기반 폴백
    if 'customer' not in response and 'research' not in response and 'text2sql' not in response:
        # 키워드 매칭
        if any(word in query for word in ['주문', '배송', '환불', '취소', '재고', '가격', '구매']):
            agent = 'customer_support'
        elif any(word in query for word in ['조사', '검색', '알아봐', '리서치', '보고서', '연구']):
            agent = 'research'
        elif any(word in query for word in ['매출', '통계', '평균', '상위', '하위', '데이터', '분석']):
            agent = 'text2sql'
        else:
            # 기본값: 질문에 숫자가 있으면 customer_support, 아니면 research
            import re
            if re.search(r'\d{5}', query):  # 주문번호 패턴
                agent = 'customer_support'
            else:
                agent = 'research'
    else:
        # LLM 응답에서 Agent 추출
        if 'customer' in response or 'support' in response:
            agent = 'customer_support'
        elif 'research' in response:
            agent = 'research'
        elif 'text2sql' in response or 'sql' in response:
            agent = 'text2sql'
        else:
            agent = 'research'  # 기본값
    
    print(f"   ➡️ 선택된 Agent: {agent}")
    
    return Command(goto=agent, update={'agent_used': agent})


print("✅ Top Supervisor 정의 완료!")

### 5.2 Agent Wrapper 노드들

각 전문 Agent를 Universal Graph에서 사용할 수 있도록 감싸줍니다.

In [ ]:
def customer_support_wrapper(state: UniversalState) -> UniversalState:
    """Customer Support Agent 실행"""
    print("\n🛒 Customer Support Agent 시작...")
    
    # Customer Support App 실행
    result = customer_support_app.invoke({
        'query': state['query'],
        'response': '',
        'agent_path': []
    })
    
    return {
        **state,
        'response': result['response'],
        'metadata': {'agent_path': result['agent_path']}
    }


def research_wrapper(state: UniversalState) -> UniversalState:
    """Research Agent 실행"""
    print("\n📚 Research Agent 시작...")
    
    # Research App 실행
    result = research_app.invoke({
        'query': state['query'],
        'wiki_content': '',
        'analysis': '',
        'report': '',
        'filename': ''
    })
    
    response = f"""✅ 리서치 보고서 생성 완료!

📄 파일: reports/{result['filename']}

미리보기:
{result['report'][:400]}...

전체 내용은 파일을 확인하세요!"""
    
    return {
        **state,
        'response': response,
        'metadata': {'filename': result['filename']}
    }


def text2sql_wrapper(state: UniversalState) -> UniversalState:
    """Text2SQL Agent 실행"""
    print("\n💾 Text2SQL Agent 시작...")
    
    # Text2SQL App 실행
    result = text2sql_app.invoke({
        'query': state['query'],
        'sql_query': '',
        'result': '',
        'formatted_response': '',
        'error': ''
    })
    
    response = f"""🔍 SQL 쿼리: {result['sql_query']}

{result['formatted_response']}"""
    
    return {
        **state,
        'response': response,
        'metadata': {'sql_query': result['sql_query']}
    }


print("✅ Agent Wrapper 노드들 정의 완료!")

---

# Part 6: Universal Graph 구축 🌐

## 💡 이 Part에서 하는 것
- 모든 Agent를 하나의 Graph로 통합
- Supervisor → Agent 라우팅 구조

**예상 시간**: 3분

In [ ]:
# Universal Graph 생성
universal_graph = StateGraph(UniversalState)

# 노드 추가
universal_graph.add_node("supervisor", top_supervisor)
universal_graph.add_node("customer_support", customer_support_wrapper)
universal_graph.add_node("research", research_wrapper)
universal_graph.add_node("text2sql", text2sql_wrapper)

# 엣지 설정
universal_graph.add_edge(START, "supervisor")
# Supervisor가 Command로 라우팅하므로 conditional edge 불필요
universal_graph.add_edge("customer_support", END)
universal_graph.add_edge("research", END)
universal_graph.add_edge("text2sql", END)

# 컴파일
universal_app = universal_graph.compile()

print("✅ Universal Graph 구축 완료!")
print("   통합된 Agent: Customer Support, Research, Text2SQL")

### 6.1 통합 시스템 테스트

In [ ]:
print("=" * 60)
print("🎯 Universal AI Assistant 통합 테스트")
print("=" * 60)

# 테스트 1: Customer Support
print("\n[테스트 1] Customer Support")
result1 = universal_app.invoke({
    'query': '주문 12345 어디까지 왔어?',
    'response': '',
    'agent_used': '',
    'metadata': {}
})
print(f"\n사용된 Agent: {result1['agent_used']}")
print(f"답변:\n{result1['response']}")

# 테스트 2: Research
print("\n" + "=" * 60)
print("[테스트 2] Research")
result2 = universal_app.invoke({
    'query': '양자컴퓨팅에 대해 조사해줘',
    'response': '',
    'agent_used': '',
    'metadata': {}
})
print(f"\n사용된 Agent: {result2['agent_used']}")
print(f"답변:\n{result2['response'][:300]}...")

# 테스트 3: Text2SQL
print("\n" + "=" * 60)
print("[테스트 3] Text2SQL")
result3 = universal_app.invoke({
    'query': '판매량 상위 3개 상품 알려줘',
    'response': '',
    'agent_used': '',
    'metadata': {}
})
print(f"\n사용된 Agent: {result3['agent_used']}")
print(f"답변:\n{result3['response']}")

print("\n" + "=" * 60)
print("✅ 모든 Agent가 정상 작동합니다!")
print("=" * 60)

---

# Part 7: Gradio 채팅 UI 🖥️

## 💡 이 Part에서 하는 것
- 사용자 친화적인 채팅 인터페이스
- Universal Agent와 연동
- 실시간 대화형 AI 시스템

**예상 시간**: 5분

In [ ]:
import gradio as gr

def chat_function(message, history):
    """Gradio 채팅 함수"""
    try:
        # Universal App 실행
        result = universal_app.invoke({
            'query': message,
            'response': '',
            'agent_used': '',
            'metadata': {}
        })
        
        # 답변에 Agent 정보 추가
        agent_emoji = {
            'customer_support': '🛒',
            'research': '📚',
            'text2sql': '💾'
        }
        
        emoji = agent_emoji.get(result['agent_used'], '🤖')
        response = f"{emoji} **{result['agent_used'].upper()}**\n\n{result['response']}"
        
        return response
    except Exception as e:
        return f"❌ 오류 발생: {str(e)}"


# Gradio Interface 생성 (호환성 개선)
demo = gr.ChatInterface(
    fn=chat_function,
    title="🎯 Universal AI Assistant",
    description="""3가지 전문 Agent가 협력하는 통합 AI 시스템
    
🛒 **Customer Support**: 주문 조회, 환불, 재고 확인
📚 **Research**: Wikipedia 리서치 보고서
💾 **Text2SQL**: 데이터 분석 쿼리

**예시 질문:**
- "주문 12345 어디까지 왔어?"
- "인공지능 역사에 대해 조사해줘"
- "매출 상위 5개 상품 알려줘"
""",
    examples=[
        "주문 12345 상태 알려줘",
        "키보드 재고 있어?",
        "블록체인 기술 조사해줘",
        "매출 상위 5개 상품은?",
        "주변기기 평균 가격은?"
    ],
    theme=gr.themes.Soft()
)

print("✅ Gradio UI 준비 완료!")

In [ ]:
# Gradio UI 실행
demo.launch(
    share=False,
    server_port=7860,
    show_error=True
)

---

# Part 8: 종합 테스트 & 마무리 🎓

## 💡 이 Part에서 하는 것
- 다양한 시나리오 테스트
- 실습 퀴즈 (3개)
- Day 3 전체 정리

**예상 시간**: 10분

### 8.1 다양한 시나리오 테스트

In [ ]:
# 다양한 질문으로 시스템 테스트
test_queries = [
    "주문 12347 배송 상태 알려줘",
    "헤드폰 재고 확인해줘",
    "주문 12346 환불하고 싶어",
    "머신러닝 기술에 대해 조사해줘",
    "재고가 가장 많은 상품 3개는?",
    "컴퓨터 카테고리 상품들 보여줘"
]

print("=" * 70)
print("🧪 종합 시나리오 테스트")
print("=" * 70)

for i, query in enumerate(test_queries, 1):
    print(f"\n[{i}/{len(test_queries)}] 질문: {query}")
    
    result = universal_app.invoke({
        'query': query,
        'response': '',
        'agent_used': '',
        'metadata': {}
    })
    
    print(f"선택된 Agent: {result['agent_used']}")
    print(f"답변 (처음 150자): {result['response'][:150]}...")
    print("-" * 70)

print("\n✅ 모든 시나리오 테스트 완료!")

---

### 🎯 Quiz 1: Supervisor 라우팅 로직 개선

**문제**: 현재 `top_supervisor` 함수는 키워드 매칭에 의존합니다. 
LLM 응답을 더 robust하게 파싱하는 개선된 버전을 작성하세요.

**요구사항**:
1. LLM 응답에서 정확히 `customer_support`, `research`, `text2sql` 중 하나를 추출
2. 응답이 불명확하면 질문 내용 기반 폴백 사용
3. 디버그 정보 출력

In [ ]:
def improved_supervisor(state: UniversalState) -> Command:
    """개선된 Supervisor - TODO: 여기를 수정하세요! 👇"""
    query = state['query']
    
    # TODO: LLM 응답을 더 robust하게 파싱하세요
    # 힌트 1: 정규표현식으로 Agent 이름 찾기
    # 힌트 2: 찾지 못하면 키워드 폴백
    # 힌트 3: 각 단계의 디버그 정보 출력
    
    agent = 'research'  # 기본값
    
    return Command(goto=agent, update={'agent_used': agent})


# 테스트
# test_state = {'query': '주문 12345 확인', 'response': '', 'agent_used': '', 'metadata': {}}
# result = improved_supervisor(test_state)
# print(f"선택된 Agent: {result.update['agent_used']}")

<details>
<summary><b>📝 정답 보기</b></summary>

```python
def improved_supervisor(state: UniversalState) -> Command:
    """개선된 Supervisor"""
    import re
    
    query = state['query']
    print(f"\n🎯 Improved Supervisor")
    print(f"   질문: {query}")
    
    # LLM 호출
    prompt = f"""다음 질문을 처리할 Agent를 선택하세요.

질문: {query}

Agent:
- customer_support: 주문, 환불, 재고
- research: Wikipedia 조사
- text2sql: 데이터 분석

답변: (Agent 이름만)"""
    
    response = llm.invoke(prompt).strip().lower()
    print(f"   LLM 응답: '{response}'")
    
    # 정규표현식으로 정확히 매칭
    agents = ['customer_support', 'research', 'text2sql']
    for agent_name in agents:
        if re.search(rf'\b{agent_name}\b', response):
            agent = agent_name
            print(f"   ✅ 매칭 성공: {agent}")
            return Command(goto=agent, update={'agent_used': agent})
    
    # 폴백: 키워드 기반
    print("   ⚠️ LLM 파싱 실패, 키워드 폴백 사용")
    if any(w in query for w in ['주문', '배송', '환불', '재고']):
        agent = 'customer_support'
    elif any(w in query for w in ['조사', '리서치', '알아봐']):
        agent = 'research'
    elif any(w in query for w in ['매출', '통계', '평균', '상위']):
        agent = 'text2sql'
    else:
        agent = 'research'
    
    print(f"   ➡️ 폴백 결과: {agent}")
    return Command(goto=agent, update={'agent_used': agent})
```

**핵심 개선사항**:
1. **정규표현식**: `\b{agent_name}\b`로 정확한 단어 매칭
2. **디버그 출력**: 각 단계의 결정 과정 확인 가능
3. **2단계 폴백**: LLM 파싱 실패 시 키워드 기반으로 전환

</details>

---

### 🎯 Quiz 2: 새로운 Agent 추가하기

**문제**: **Math Agent**를 추가하세요. 수학 계산 질문을 처리합니다.

**요구사항**:
1. `MathState` 정의 (query, expression, result)
2. `calculate_node` 구현 (eval 사용 또는 LLM)
3. Math Graph 구축
4. Universal Graph에 통합

**예시**: "123 곱하기 456은?" → "56088"

In [ ]:
# TODO: 여기를 수정하세요! 👇

# 1. State 정의
class MathState(TypedDict):
    pass  # TODO: query, expression, result 정의

# 2. Calculate 노드
def calculate_node(state: MathState) -> MathState:
    """수학 계산 노드"""
    # TODO: 구현하세요
    # 힌트 1: LLM에게 수식 추출 요청
    # 힌트 2: eval() 또는 LLM으로 계산
    # 힌트 3: 안전을 위해 eval은 주의해서 사용
    return state

# 3. Graph 구축
# math_graph = StateGraph(MathState)
# TODO: 노드와 엣지 추가
# math_app = math_graph.compile()

# 4. Wrapper
def math_wrapper(state: UniversalState) -> UniversalState:
    """Math Agent Wrapper"""
    # TODO: math_app 실행 및 결과 반환
    return state

# 5. Supervisor에 추가
# TODO: top_supervisor에 'math' 선택 로직 추가
# TODO: Universal Graph에 math_wrapper 노드 추가

print("TODO: Math Agent를 구현하세요!")

<details>
<summary><b>📝 정답 보기</b></summary>

```python
import re

# 1. State 정의
class MathState(TypedDict):
    query: str
    expression: str
    result: str

# 2. Calculate 노드
def calculate_node(state: MathState) -> MathState:
    """수학 계산"""
    query = state['query']
    
    # LLM에게 수식 추출 요청
    prompt = f"""다음 질문에서 수학 수식을 Python 표현식으로 변환하세요.

질문: {query}

예시:
"123 곱하기 456" → "123 * 456"
"100 더하기 50 빼기 30" → "100 + 50 - 30"

수식만 출력하세요:"""
    
    expression = llm.invoke(prompt).strip()
    
    # 안전한 eval (숫자와 연산자만 허용)
    allowed_chars = set('0123456789+-*/(). ')
    if all(c in allowed_chars for c in expression):
        try:
            result = eval(expression)
            return {**state, 'expression': expression, 'result': str(result)}
        except:
            return {**state, 'expression': expression, 'result': '계산 오류'}
    else:
        return {**state, 'expression': expression, 'result': '잘못된 수식'}

# 3. Graph 구축
math_graph = StateGraph(MathState)
math_graph.add_node("calculate", calculate_node)
math_graph.add_edge(START, "calculate")
math_graph.add_edge("calculate", END)
math_app = math_graph.compile()

# 4. Wrapper
def math_wrapper(state: UniversalState) -> UniversalState:
    result = math_app.invoke({'query': state['query'], 'expression': '', 'result': ''})
    response = f"🔢 계산: {result['expression']} = {result['result']}"
    return {**state, 'response': response, 'metadata': {'expression': result['expression']}}

# 5. Supervisor 수정
# top_supervisor에서:
# if any(w in query for w in ['계산', '더하기', '빼기', '곱하기', '나누기']):
#     agent = 'math'

# Universal Graph에:
# universal_graph.add_node("math", math_wrapper)
# universal_graph.add_edge("math", END)
```

**테스트**:
```python
result = math_app.invoke({'query': '123 곱하기 456', 'expression': '', 'result': ''})
print(result)  # {'query': '123 곱하기 456', 'expression': '123 * 456', 'result': '56088'}
```

</details>

---

### 🎯 Quiz 3: Multi-Step Reasoning 구현

**문제**: Customer Support Agent가 환불 전에 먼저 주문을 확인하도록 개선하세요.

**요구사항**:
1. Refund Agent가 실행되기 전에 Order Agent 먼저 실행
2. 주문이 존재하면 → Refund 진행
3. 주문이 없으면 → 에러 메시지
4. **Conditional Routing** 사용 (03번 노트북 개념)

**힌트**: `add_conditional_edges` 또는 `Command(goto=...)`

In [ ]:
# TODO: 여기를 수정하세요! 👇

class ImprovedCSState(TypedDict):
    query: str
    order_exists: bool  # 주문 존재 여부
    response: str

def check_order_node(state: ImprovedCSState) -> ImprovedCSState:
    """주문 확인 노드"""
    # TODO: 주문 존재 여부 확인
    # 힌트: search_order 사용, "찾을 수 없습니다" 체크
    return state

def refund_decision(state: ImprovedCSState) -> str:
    """환불 진행 여부 결정"""
    # TODO: order_exists가 True면 'proceed', False면 'error'
    return 'error'

def process_refund_node(state: ImprovedCSState) -> ImprovedCSState:
    """환불 처리"""
    # TODO: process_refund 실행
    return state

def error_node(state: ImprovedCSState) -> ImprovedCSState:
    """에러 처리"""
    return {**state, 'response': '주문을 찾을 수 없어 환불할 수 없습니다.'}

# Graph 구축
# improved_cs_graph = StateGraph(ImprovedCSState)
# TODO: 노드 추가 및 conditional routing 설정

print("TODO: Multi-Step Reasoning을 구현하세요!")

<details>
<summary><b>📝 정답 보기</b></summary>

```python
import re

class ImprovedCSState(TypedDict):
    query: str
    order_exists: bool
    response: str

def check_order_node(state: ImprovedCSState) -> ImprovedCSState:
    """주문 존재 여부 확인"""
    query = state['query']
    order_id_match = re.search(r'\d{5}', query)
    
    if order_id_match:
        order_id = order_id_match.group()
        result = search_order(order_id)
        exists = "찾을 수 없습니다" not in result
    else:
        exists = False
    
    print(f"✅ 주문 확인: {'존재' if exists else '없음'}")
    return {**state, 'order_exists': exists}

def refund_decision(state: ImprovedCSState) -> str:
    """환불 진행 여부 결정"""
    return 'proceed' if state['order_exists'] else 'error'

def process_refund_node(state: ImprovedCSState) -> ImprovedCSState:
    """환불 처리"""
    query = state['query']
    order_id = re.search(r'\d{5}', query).group()
    result = process_refund(order_id, "고객 요청")
    return {**state, 'response': result}

def error_node(state: ImprovedCSState) -> ImprovedCSState:
    """에러 처리"""
    return {**state, 'response': '⚠️ 주문을 찾을 수 없어 환불할 수 없습니다.'}

# Graph 구축
improved_cs_graph = StateGraph(ImprovedCSState)
improved_cs_graph.add_node("check_order", check_order_node)
improved_cs_graph.add_node("process_refund", process_refund_node)
improved_cs_graph.add_node("error", error_node)

improved_cs_graph.add_edge(START, "check_order")
improved_cs_graph.add_conditional_edges(
    "check_order",
    refund_decision,
    {'proceed': 'process_refund', 'error': 'error'}
)
improved_cs_graph.add_edge("process_refund", END)
improved_cs_graph.add_edge("error", END)

improved_cs_app = improved_cs_graph.compile()
```

**테스트**:
```python
# 성공 케이스
result1 = improved_cs_app.invoke({'query': '주문 12345 환불', 'order_exists': False, 'response': ''})
print(result1['response'])  # 환불 처리 완료

# 실패 케이스
result2 = improved_cs_app.invoke({'query': '주문 99999 환불', 'order_exists': False, 'response': ''})
print(result2['response'])  # 주문을 찾을 수 없어 환불할 수 없습니다
```

**핵심 개념**:
- **Conditional Routing**: `add_conditional_edges`로 조건부 분기
- **Multi-Step**: 주문 확인 → 조건 판단 → 환불 or 에러
- **State 공유**: `order_exists` 필드로 노드 간 정보 전달

</details>

---

# 🎓 Day 3 전체 정리

## 📚 배운 개념들

| 노트북 | 핵심 개념 | 적용 위치 (08번) |
|--------|----------|------------------|
| **00-01** | StateGraph 기초 | 모든 Graph 구조 |
| **00-02** | 조건부 라우팅 | Supervisor, CS Router |
| **01-01** | Fan-out/Fan-in | (미사용, 개념만) |
| **02-01** | ReAct 패턴 | Text2SQL Agent |
| **03-01** | Multi-Agent Handoffs | Customer Support |
| **04** | ReAct Tool Loop | (간소화 버전) |
| **05** | **Supervisor 패턴** | **Top Supervisor** ⭐ |
| **06** | MCP 통합 | (Wikipedia 도구) |
| **07** | Memory & Persistence | (미적용, 선택 사항) |

---

## 🏗️ Ultimate Integration 아키텍처

```
        사용자 (Gradio UI)
             ↓
    Top Supervisor (LLM)
       /      |      \
      /       |       \
  Customer  Research  Text2SQL
  Support    Agent     Agent
     ↓         ↓         ↓
  3개 노드   3개 노드   3개 노드
  (Router)  (Sequential) (Sequential)
```

**총 Graph 개수**: 4개 (CS + Research + Text2SQL + Universal)

**총 노드 개수**: 15개
- Top Supervisor: 4개 (supervisor + 3 wrappers)
- Customer Support: 4개 (router + 3 전문가)
- Research: 3개 (search + analyze + write)
- Text2SQL: 3개 (generate + execute + format)

---

## 🚀 실무 활용 팁

### 1. Memory 추가하기 (07번 노트북)
```python
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
universal_app = universal_graph.compile(checkpointer=memory)

# 사용 시:
config = {"configurable": {"thread_id": "user_123"}}
result = universal_app.invoke(state, config=config)
```

### 2. Error Handling 개선
```python
try:
    result = universal_app.invoke(state)
except Exception as e:
    # Fallback Agent 실행
    result = {'response': f"시스템 오류: {e}", 'agent_used': 'error_handler'}
```

### 3. Streaming 응답 (실시간)
```python
for chunk in universal_app.stream(state):
    print(chunk)  # 노드별 진행 상황 실시간 출력
```

### 4. Production 배포
- **API 서버**: FastAPI + LangGraph
- **모니터링**: LangSmith 추가
- **캐싱**: Redis로 LLM 응답 캐싱
- **비동기**: `ainvoke()` 사용

---

## 🎯 다음 단계 학습 추천

1. **LangGraph Cloud**: 클라우드 배포 및 스케일링
2. **Multi-Agent 협업**: 여러 Agent가 동시에 작업
3. **Human-in-the-Loop 고급**: 승인 플로우, 피드백 반영
4. **RAG + Agent 통합**: Day 2 RAG + Day 3 Agent 결합
5. **Custom Tools**: 외부 API, 데이터베이스 연동

---

## 🎉 축하합니다!

Day 3 **LangGraph Multi-Agent 시스템**을 완성했습니다!

**달성한 것들**:
- ✅ StateGraph 기초부터 고급까지
- ✅ 3가지 전문 Agent 구현
- ✅ Supervisor 패턴으로 통합
- ✅ Gradio UI로 사용자 경험 완성
- ✅ 실습 퀴즈로 이해도 검증

**Day 1-3 전체 여정**:
- Day 1: Fine-tuning (모델 개선)
- Day 2: RAG (지식 검색)
- Day 3: **Agent (작업 자동화)** ← 완료!

이제 여러분은 **Production-Ready AI 시스템**을 만들 수 있습니다! 🚀

---

*생성일: 2025-10-26*  
*노트북 08: Ultimate Integration*  
*총 셀 개수: ~70개*
